# Week8 Advanced
- LLaMA 3.2 모델의 양자화에 따른 GPU 사용량 비교

##  개요
- 모델: **Bllossom/llama-3.2-Korean-Bllossom-3B** 
- 위 모델을 로드하여 비양자화, 8비트, 4비트로 로드
- gpu 사용량 분석하여 메모리 절약 효과를 분석석  

##  코드 실행 흐름
1. **비양자화 모델 로드 및 추론 수행**
   - `bfloat16` 모델을 로드하고 **GPU 사용량**을 측정
   - 프롬프트를 입력받아 **추론 후 GPU 사용량**을 기록

2. **8비트 양자화 모델 로드 및 추론 수행**
   - `load_in_8bit=True` 옵션을 사용하여 8비트 양자화 모델 로드
   - **모델 로드 시 GPU 사용량 감소 효과 확인**
   - 동일한 프롬프트로 추론하고 **GPU 사용량 기록**

3. **4비트 양자화 모델 로드 및 추론 수행**
   - `load_in_4bit=True` 옵션을 사용하여 4비트 양자화 모델 로드
   - **가장 적은 GPU 메모리를 사용하면서도 모델을 실행 가능**
   - 동일한 프롬프트로 추론 후 **GPU 사용량 기록**


In [ ]:
import os
import torch
import wandb
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

def get_gpu_memory():
    """GPU 메모리 사용량 반환 (MB 단위)"""
    torch.cuda.synchronize()
    mem_used = torch.cuda.memory_allocated() / (1024 ** 2)
    peak_mem_used = torch.cuda.max_memory_allocated() / (1024 ** 2)
    return mem_used, peak_mem_used

def run_inference(model, tokenizer, prompt):
    """모델 추론 및 GPU 메모리 사용량 측정"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    torch.cuda.reset_peak_memory_stats()
    with torch.no_grad():
        output = model.generate(**inputs, max_length=1024)

    mem_used, peak_mem_used = get_gpu_memory()
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return output_text, mem_used, peak_mem_used

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
tokenizer_base = AutoTokenizer.from_pretrained(model_id)

torch.cuda.reset_peak_memory_stats()
model_base = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

mem_base, peak_mem_base = get_gpu_memory()
print(f"양자화 전 모델 GPU 사용량: {mem_base:.2f}MB (최대 {peak_mem_base:.2f}MB)", end="\n\n")

prompt = "서울의 관광코스를 짜고 싶습니다. 외국인이 왔을 때 역사적으로 많은 경험을 얻을 수 있도록 설계해주세요"
output_base, inf_mem_base, peak_inf_mem_base = run_inference(model_base, tokenizer_base, prompt)
print(f"[양자화 전] 모델 출력: {output_base}", end="\n\n")
print(f"[양자화 전] 추론 시 GPU 사용량: {inf_mem_base:.2f}MB (최대 {peak_inf_mem_base:.2f}MB)", end="\n\n")

del model_base
torch.cuda.empty_cache()

tokenizer_8bit = AutoTokenizer.from_pretrained(model_id)

torch.cuda.reset_peak_memory_stats()
model_8bit = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto",
)

mem_8bit, peak_mem_8bit = get_gpu_memory()
print(f"8비트 양자화 후 모델 GPU 사용량: {mem_8bit:.2f}MB (최대 {peak_mem_8bit:.2f}MB)", end="\n\n")

output_8bit, inf_mem_8bit, peak_inf_mem_8bit = run_inference(model_8bit, tokenizer_8bit, prompt)
print(f"[8비트 양자화] 모델 출력: {output_8bit}", end="\n\n")
print(f"[8비트 양자화] 추론 시 GPU 사용량: {inf_mem_8bit:.2f}MB (최대 {peak_inf_mem_8bit:.2f}MB)", end="\n\n")

del model_8bit
torch.cuda.empty_cache()

quantization_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tokenizer_4bit = AutoTokenizer.from_pretrained(model_id)

torch.cuda.reset_peak_memory_stats()
model_4bit = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config_4bit,
    device_map="auto",
)


mem_4bit, peak_mem_4bit = get_gpu_memory()
print(f"4비트 양자화 후 모델 GPU 사용량: {mem_4bit:.2f}MB (최대 {peak_mem_4bit:.2f}MB)", end="\n\n")

output_4bit, inf_mem_4bit, peak_inf_mem_4bit = run_inference(model_4bit, tokenizer_4bit, prompt)
print(f"[4비트 양자화] 모델 출력: {output_4bit}", end="\n\n")
print(f"[4비트 양자화] 추론 시 GPU 사용량: {inf_mem_4bit:.2f}MB (최대 {peak_inf_mem_4bit:.2f}MB)", end="\n\n")

del model_4bit
torch.cuda.empty_cache()


print("📊 최종 비교 결과:")
print(f"🔹 모델 로드 후 GPU 절약량 (8비트): {mem_base - mem_8bit:.2f}MB (최대 {peak_mem_base - peak_mem_8bit:.2f}MB)")
print(f"🔹 모델 로드 후 GPU 절약량 (4비트): {mem_base - mem_4bit:.2f}MB (최대 {peak_mem_base - peak_mem_4bit:.2f}MB)")
print(f"🔹 모델 추론 시 GPU 절약량 (8비트): {inf_mem_base - inf_mem_8bit:.2f}MB (최대 {peak_inf_mem_base - peak_inf_mem_8bit:.2f}MB)")
print(f"🔹 모델 추론 시 GPU 절약량 (4비트): {inf_mem_base - inf_mem_4bit:.2f}MB (최대 {peak_inf_mem_base - peak_inf_mem_4bit:.2f}MB)")



Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.05s/it]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


양자화 전 모델 GPU 사용량: 6128.33MB (최대 6128.33MB)

[양자화 전] 모델 출력: 서울의 관광코스를 짜고 싶습니다. 외국인이 왔을 때 역사적으로 많은 경험을 얻을 수 있도록 설계해주세요. 서울의 주요 관광지와 그들의 역사적 의미를 소개합니다.

### 1. 경복궁
경복궁은 조선 시대의 왕궁으로, 1395년 왕조 창건 후 1398년까지 13대 왕이 재위한 곳입니다. 경복궁은 조선 시대의 문화와 역사, 아름다움을 한곳으로, 왕궁의 전통적인 건축물과 정원, 궁궐의 내부를 경험할 수 있습니다.

### 2. 성북대성전
성북대성전은 조선 중기(15세기)에 세워진 사찰로, 조선 시대의 사찰 중에서 가장 큰 규모와 아름다움을 자랑합니다. 성북대성전은 조선의 역사와 문화를 이해할 수 있는 중요한 곳입니다.

### 3. 경복궁 정원
경복궁 정원은 조선 시대의 정원 건축 기술과 아름다움을 보여주는 곳입니다. 정원에는 다양한 식물과 인공수목이 자리 잡고 있으며, 정원 속에 있는 수영장, 정원 경로 등도 관람할 수 있습니다.

### 4. 인사당
인사당은 조선 중기(15세기)에 세워진 사찰로, 조선 시대의 사찰 중에서 가장 아름다운 건축물 중 하나입니다. 인사당은 정원 속에 위치해 있으며, 정원 속의 수영장과 수영장 주변의 신라 유적이 관람할 수 있습니다.

### 5. 서울시립박물관
서울시립박물관은 서울에서 가장 큰 박물관으로, 조선 시대부터 현대까지의 서울의 역사와 문화를 전시한 곳입니다. 박물관에는 다양한 전시물과 전시회가 열리고 있으며, 박물관 내에 있는 정원과 공원도 관람할 수 있습니다.

### 6. 청계천
청계천은 조선 시대의 주변 지역에 위치한 강으로, 서울의 역사와 문화를 이해할 수 있는 중요한 곳입니다. 청계천은 조선 시대의 상인들이 활동한 곳이었으며, 현재는 서울의 중심지로 자리 잡고 있습니다.

### 7. 성북대성전 정원
성북대성전 정원은 조선 중기(15세기)에 세워진 사찰로, 조선 시대의 정원 건축 기술과 아름다움을 보여주는 곳입니

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.14s/it]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


8비트 양자화 후 모델 GPU 사용량: 3451.41MB (최대 3475.37MB)

[8비트 양자화] 모델 출력: 서울의 관광코스를 짜고 싶습니다. 외국인이 왔을 때 역사적으로 많은 경험을 얻을 수 있도록 설계해주세요. 서울의 대표적인 명소와 함께 다양한 문화를 경험할 수 있는 방법을 제안해 주세요.

### 서울의 대표적인 명소와 문화 경험 방법

1. **경복궁**
   - ** 역사적 경험**: 한국의 역사와 문화를 이해할 수 있는 장소입니다. 경복궁은 조선 시대에 남아 있는 유일한 궁궐로, 건축물과 문물, 역사적인 장소들을 감상할 수 있습니다.
   - **문화 경험**: 궁궐 내부에서 다양한 문화적 경험을 할 수 있습니다. 예를 들어, 궁궐 내에서 진행되는 전통 공연을 감상하거나, 전통 의상으로 상装을 입고 궁궐을 산책할 수 있습니다.

2. **정성동의 전통시장**
   - **문화 경험**: 전통시장에서 다양한 전통 음식과 소품을 맛보며, 전통 의상을 입고 전통 공연을 감상할 수 있습니다. 전통시장은 한국의 전통과 문화를 체험할 수 있는 좋은 장소입니다.
   - **명소**: 정성동의 전통시장에서 다양한 전통 음식을 맛보며, 전통 의상을 입고 전통 공연을 감상할 수 있습니다.

3. **한강**
   - **명소**: 서울의 대표적인 명소 중 하나로, 한강을 따라 걷거나 보트를 타며 서울의 풍경을 감상할 수 있습니다.
   - **문화 경험**: 한강에 위치한 전통 공원인 한강공원에서 전통 공연을 감상하거나, 한강을 따라 걷면서 서울의 문화를 체험할 수 있습니다.

4. **남대문시장**
   - **문화 경험**: 전통시장에서 다양한 전통 음식을 맛보며, 전통 의상을 입고 전통 공연을 감상할 수 있습니다.
   - **명소**: 남대문시장은 서울의 대표적인 전통시장 중 하나로, 다양한 전통 음식을 맛보며 전통 문화를 체험할 수 있습니다.

5. **동대문디자인플라자**
   - **문화 경험**: 다양한 전통과 현대 문화를 체험할 수 있는 공

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.31s/it]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


4비트 양자화 후 모델 GPU 사용량: 2158.84MB (최대 2195.94MB)

[4비트 양자화] 모델 출력: 서울의 관광코스를 짜고 싶습니다. 외국인이 왔을 때 역사적으로 많은 경험을 얻을 수 있도록 설계해주세요. 서울의 역사적 Sites를 순회하며, 서울의 문화와 역사적 상징을 깊이 이해해보세요. 

1.  **고려시대( 고려, 고려시대, 고려왕국)**

    *   **고려의 역사적 sites** : 고려의 역사적 sites를 방문하여 고려시대의 역사와 문화를 깊이 이해해보세요.
    *   **고려의 문화적 상징** : 고려의 문화적 상징, 예를 들어 고려의 전통 의상, 고려의 전통 음악, 고려의 전통 전시회를 방문하여 고려의 문화를 깊이 이해해보세요.

2.  **중세시대(고려, 고려시대, 고려왕국)**

    *   **고려의 역사적 sites** : 고려의 역사적 sites를 방문하여 중세시대의 역사와 문화를 깊이 이해해보세요.
    *   **고려의 문화적 상징** : 고려의 문화적 상징, 예를 들어 고려의 전통 의상, 고려의 전통 음악, 고려의 전통 전시회를 방문하여 고려의 문화를 깊이 이해해보세요.

3.  **현대시대(서울, 고려, 고려시대)**

    *   **서울의 역사적 sites** : 서울의 역사적 sites를 방문하여 현대시대의 역사와 문화를 깊이 이해해보세요.
    *   **서울의 문화적 상징** : 서울의 문화적 상징, 예를 들어 서울의 전통 의상, 서울의 전통 음악, 서울의 전통 전시회를 방문하여 서울의 문화를 깊이 이해해보세요.

### 서울의 역사적 Sites

1.  **고려시대(고려, 고려시대, 고려왕국)**

    *   **고려의 역사적 sites** : 고려의 역사적 sites를 방문하여 고려시대의 역사와 문화를 깊이 이해해보세요. 

    *   **고려의 역사적 sites** : 고려의 역사적 sites를 방문하여 고려시대의 역사와 문화를 깊이 이해해보세요. 

2.  **중세시대(고려

## 결론
- 4비트 양자화가 가장 큰 GPU 메모리 절약 효과를 보임
- 8비트 양자화도 효과적이지만, 4비트보다는 메모리를 더 많이 사용
- 양자화를 할 수록 답변의 퀄리티가 떨어지다가, 4비트에서부터는 대답이 망가진 것을 확인
- 이후 챗봇에 엮어서 진행할 예정

In [4]:
import pandas as pd

# 데이터 정리
data = {
    "양자화 방식": ["비양자화", "8비트 양자화", "4비트 양자화"],
    "모델 로드 GPU 사용량 (MB)": [mem_base, mem_8bit, mem_4bit],
    "모델 로드 최대 GPU 사용량 (MB)": [peak_mem_base, peak_mem_8bit, peak_mem_4bit],
    "추론 GPU 사용량 (MB)": [inf_mem_base, inf_mem_8bit, inf_mem_4bit],
    "추론 최대 GPU 사용량 (MB)": [peak_inf_mem_base, peak_inf_mem_8bit, peak_inf_mem_4bit],
}

# 데이터프레임 생성
df = pd.DataFrame(data)

df


,양자화 방식,모델 로드 GPU 사용량 (MB),모델 로드 최대 GPU 사용량 (MB),추론 GPU 사용량 (MB),추론 최대 GPU 사용량 (MB)
0,비양자화,6128.334473,6128.334473,6136.468262,6262.069336
1,8비트 양자화,3451.412598,3475.369141,3453.213379,3579.779297
2,4비트 양자화,2158.835449,2195.937500,2158.844238,2284.437500


챗봇에서 아래와 같은 형태로 4비트 양자화를 하여 모델을 구현했습니다.

```python
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
)

custom_pipeline = KoreanLlamaPipeline(model, tokenizer)
llm = KoreanLlamaLangChainLLM(pipeline=custom_pipeline)
```